# Sample Notebook for Zero-Shot Inference with CheXzero
This notebook walks through how to use CheXzero to perform zero-shot inference on a chest x-ray image dataset.

## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Optional

import sys
sys.path.append('../')

from eval import evaluate
from zero_shot import make, make_true_labels, run_softmax_eval

%load_ext autoreload
%autoreload 2

## Preprocessing Data

In [2]:
from preprocess_padchest import get_paths, img_to_h5
try:
    data_img_path = '../data/images/' 
    image_paths = get_paths(data_img_path)
    print('Data already downloaded....')
except ValueError as e:
    print(e)

Data already downloaded....


In [3]:
output_path = Path('../data/chexpert_test.h5')
img_to_h5(image_paths, output_path)

  0%|          | 0/137 [00:00<?, ?it/s]

Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 failed loading...
Image 0 fail

[]

## Directories and Constants

In [4]:
## Define Zero Shot Labels and Templates

# ----- DIRECTORIES ------ #
cxr_filepath: str = '../data/chexpert_test.h5' # filepath of chest x-ray images (.h5)
cxr_true_labels_path: Optional[str] = '../data/groundtruth.csv' # (optional for evaluation) if labels are provided, provide path
model_dir: str = '../checkpoints/chexzero_weights/' # where pretrained models are saved (.pt) 
predictions_dir: Path = Path('../predictions') # where to save predictions
cache_dir: str = predictions_dir / "cached" # where to cache ensembled predictions

context_length: int = 77

# ------- LABELS ------  #
# Define labels to query each image | will return a prediction for each label
cxr_labels: List[str] = ['Atelectasis','Cardiomegaly', 
                                      'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
                                      'Lung Opacity', 'No Finding','Pleural Effusion', 'Pleural Other', 'Pneumonia', 
                                      'Pneumothorax', 'Support Devices']

# ---- TEMPLATES ----- # 
# Define set of templates | see Figure 1 for more details                        
cxr_pair_template: Tuple[str] = ("{}", "no {}")

# ----- MODEL PATHS ------ #
# If using ensemble, collect all model paths
model_paths = []
for subdir, dirs, files in os.walk(model_dir):
    for file in files:
        full_dir = os.path.join(subdir, file)
        model_paths.append(full_dir)
        
print(model_paths)

['../checkpoints/chexzero_weights/best_128_0.0002_original_15000_0.859.pt', '../checkpoints/chexzero_weights/best_128_0.0002_original_8000_0.857.pt', '../checkpoints/chexzero_weights/best_128_5e-05_original_22000_0.855.pt', '../checkpoints/chexzero_weights/best_64_0.0001_original_16000_0.861.pt', '../checkpoints/chexzero_weights/best_64_0.0001_original_17000_0.863.pt', '../checkpoints/chexzero_weights/best_64_0.0001_original_35000_0.864.pt', '../checkpoints/chexzero_weights/best_64_0.0002_original_23000_0.854.pt', '../checkpoints/chexzero_weights/best_64_5e-05_original_16000_0.858.pt', '../checkpoints/chexzero_weights/best_64_5e-05_original_18000_0.862.pt', '../checkpoints/chexzero_weights/best_64_5e-05_original_22000_0.864.pt']


## Run Inference

In [5]:
## Run the model on the data set using ensembled models
def ensemble_models(
    model_paths: List[str], 
    cxr_filepath: str, 
    cxr_labels: List[str], 
    cxr_pair_template: Tuple[str], 
    cache_dir: str = None, 
    save_name: str = None,
) -> Tuple[List[np.ndarray], np.ndarray]: 
    """
    Given a list of `model_paths`, ensemble model and return
    predictions. Caches predictions at `cache_dir` if location provided.

    Returns a list of each model's predictions and the averaged
    set of predictions.
    """

    predictions = []
    model_paths = sorted(model_paths) # ensure consistency of 
    for path in model_paths: # for each model
        model_name = Path(path).stem

        # load in model and `torch.DataLoader`
        model, loader = make(
            model_path=path, 
            cxr_filepath=cxr_filepath, 
        ) 
        
        # path to the cached prediction
        if cache_dir is not None:
            if save_name is not None: 
                cache_path = Path(cache_dir) / f"{save_name}_{model_name}.npy"
            else: 
                cache_path = Path(cache_dir) / f"{model_name}.npy"

        # if prediction already cached, don't recompute prediction
        if cache_dir is not None and os.path.exists(cache_path): 
            print("Loading cached prediction for {}".format(model_name))
            y_pred = np.load(cache_path)
        else: # cached prediction not found, compute preds
            print("Inferring model {}".format(path))
            y_pred = run_softmax_eval(model, loader, cxr_labels, cxr_pair_template)
            if cache_dir is not None: 
                Path(cache_dir).mkdir(exist_ok=True, parents=True)
                np.save(file=cache_path, arr=y_pred)
        predictions.append(y_pred)
    
    # compute average predictions
    y_pred_avg = np.mean(predictions, axis=0)
    
    return predictions, y_pred_avg

In [6]:
predictions, y_pred_avg = ensemble_models(
    model_paths=model_paths, 
    cxr_filepath=cxr_filepath, 
    cxr_labels=cxr_labels, 
    cxr_pair_template=cxr_pair_template, 
    cache_dir=cache_dir,
)

Inferring model ../checkpoints/chexzero_weights/best_128_0.0002_original_15000_0.859.pt


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/137 [00:00<?, ?it/s]

...

Inferring model ../checkpoints/chexzero_weights/best_64_5e-05_original_22000_0.864.pt


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/137 [00:00<?, ?it/s]

In [7]:
# save averaged preds
pred_name = "chexpert_preds.npy" # add name of preds
predictions_dir = predictions_dir / pred_name
np.save(file=predictions_dir, arr=y_pred_avg)

In [8]:
predictions = pd.DataFrame(np.load(predictions_dir))
predictions.columns = cxr_labels
print(predictions.shape)
predictions.describe()

(137, 14)


,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
count,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.0000,137.000000,137.000000,137.00000,137.000000,137.000000,137.000000,137.000000
mean,0.498426,0.497896,0.499816,0.500577,0.498011,0.500052,0.4992,0.500516,0.499808,0.49958,0.498706,0.499921,0.497867,0.497444
std,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
min,0.498426,0.497896,0.499816,0.500577,0.498011,0.500052,0.4992,0.500516,0.499808,0.49958,0.498706,0.499921,0.497867,0.497444
25%,0.498426,0.497896,0.499816,0.500577,0.498011,0.500052,0.4992,0.500516,0.499808,0.49958,0.498706,0.499921,0.497867,0.497444
50%,0.498426,0.497896,0.499816,0.500577,0.498011,0.500052,0.4992,0.500516,0.499808,0.49958,0.498706,0.499921,0.497867,0.497444
75%,0.498426,0.497896,0.499816,0.500577,0.498011,0.500052,0.4992,0.500516,0.499808,0.49958,0.498706,0.499921,0.497867,0.497444
max,0.498426,0.497896,0.499816,0.500577,0.498011,0.500052,0.4992,0.500516,0.499808,0.49958,0.498706,0.499921,0.497867,0.497444


## (Optional) Evaluate Results
If ground truth labels are available, compute AUC on each pathology to evaluate the performance of the zero-shot model. 

In [9]:
# make test_true
# test_pred = y_pred_avg
# test_true = make_true_labels(cxr_true_labels_path=cxr_true_labels_path, cxr_labels=cxr_labels)

# evaluate model

# cxr_results = evaluate(test_true, test_pred, cxr_labels)

# boostrap evaluations for 95% confidence intervals
# bootstrap_results = bootstrap(test_pred, test_pred, cxr_labels)

In [10]:
# display AUC with confidence intervals
# bootstrap_results[1]